<a href="https://colab.research.google.com/github/DLOHai/DLQA_Torch/blob/master/10_4_ImageFolder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
!git clone https://github.com/deeplearningzerotoall/PyTorch/

fatal: destination path 'PyTorch' already exists and is not an empty directory.


In [0]:
path = './PyTorch/custom_data'

In [0]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root=path+'/train_data', transform=trans)

In [0]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        # 모델 형태 바로바로 형태 확인해서 ㄱㄱ 
        #print(out.shape) 
        out = self.layer2(out)
        # 사용 후 꼭 지워주기
        #print(out.shape) 
        out = out.view(out.shape[0], -1)
        #print(out.shape)
        out = self.layer3(out)
        return out


In [0]:
# testing
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [0]:
optimizer = optim.Adam(net.parameters(), lr=0.0005)
loss_func = nn.CrossEntropyLoss().to(device)

In [10]:
total_batch = len(data_loader)

epochs = 10

for epoch in range(epochs):
  avg_cost = 0.0
  for num, data in enumerate(data_loader):
    imgs, labels = data
    imgs = imgs.to(device)
    labels = labels.to(device)
    
    optimizer.zero_grad()
    out=net(imgs)
    loss=loss_func(out, labels)
    loss.backward()
    optimizer.step()
    
    avg_cost += loss/total_batch
    
  print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!!!!')

[Epoch: 1] cost = 0.24109964072704315
[Epoch: 2] cost = 0.001618753420189023
[Epoch: 3] cost = 0.0007355586276389658
[Epoch: 4] cost = 0.00026842154329642653
[Epoch: 5] cost = 0.00018114221165888011
[Epoch: 6] cost = 0.00013052142458036542
[Epoch: 7] cost = 8.827041165204719e-05
[Epoch: 8] cost = 8.09003395261243e-05
[Epoch: 9] cost = 5.911484549869783e-05
[Epoch: 10] cost = 5.084535587229766e-05
Learning Finished!!!!


In [0]:
# 쓸 때마다 매번 학습할 수 없다!
# 학습된 모델 저장
torch.save(net.state_dict(), "/content/model.pth")

In [0]:
new_net = CNN().to(device)

In [13]:
new_net.load_state_dict(torch.load('./model.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [14]:
# 저장 전 모델과 저장 된 모델이 같은지 확인
print(net.layer1[0])
print(new_net.layer1[0])

print("net == new_net?", net.layer1[0] == new_net.layer1[0])

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
net == new_net? False


In [15]:
# 저장 전 모델과 저장 된 모델이 같은지 확인
print("net == new_net .layer1[0].weight[0][0][0]?")
net.layer1[0].weight[0][0][0] == new_net.layer1[0].weight[0][0][0]

net == new_net .layer1[0].weight[0][0][0]?


tensor([1, 1, 1, 1, 1], device='cuda:0', dtype=torch.uint8)

In [16]:
print("net == new_net .layer1[0].weight[0]?")
net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

net == new_net .layer1[0].weight[0]?


tensor([[[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1]]], device='cuda:0', dtype=torch.uint8)

In [0]:
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root=path+'/test_data', transform=trans)

In [0]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [19]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
